In [2]:
import tkinter as tk
from tkinter import ttk, filedialog
import numpy as np
import pandas as pd
import joblib


# -------------------------- DEFINING GLOBAL VARIABLES -------------------------

selectionbar_color = '#eff5f6'
sidebar_color = '#e0b643'
header_color = '#000000'
visualisation_frame_color = "#ffffff"

# ------------------------------- ROOT WINDOW ----------------------------------


class TkinterApp(tk.Tk):

    def __init__(self):
        tk.Tk.__init__(self)
        self.title("Prediction Application")

        self.geometry("1100x700")
        self.resizable(0, 0)
        self.title('Prediction Application')
        self.config(background=selectionbar_color)
        icon = tk.PhotoImage(file='images/icon.png')
        self.iconphoto(True, icon)

        self.header = tk.Frame(self, bg=header_color)
        self.header.place(relx=0.3, rely=0, relwidth=0.7, relheight=0.1)

        self.sidebar = tk.Frame(self, bg=sidebar_color)
        self.sidebar.place(relx=0, rely=0, relwidth=0.3, relheight=1)

        self.brand_frame = tk.Frame(self.sidebar, bg=sidebar_color)
        self.brand_frame.place(relx=0, rely=0, relwidth=1, relheight=0.15)
        self.uni_logo = icon.subsample(9)
        logo = tk.Label(self.brand_frame, image=self.uni_logo, bg=sidebar_color)
        logo.place(x=5, y=20)
    
        uni_name = tk.Label(self.brand_frame,
                            text='GoBestCab',
                            bg=sidebar_color,
                            font=("", 15, "bold")
                            )
        uni_name.place(x=55, y=27, anchor="w")

        self.submenu_frame = tk.Frame(self.sidebar, bg=sidebar_color)
        self.submenu_frame.place(relx=0, rely=0.2, relwidth=1, relheight=0.85)
        submenu1 = SidebarSubMenu(self.submenu_frame,
                                  sub_menu_heading='Dangerous Trip Prediction',
                                  sub_menu_options=["MLP Classifier Model",
                                                    "Guassian NB Model",
                                                    "Logistic Regression Model",
                                                    ]
                                  )
        submenu1.options["MLP Classifier Model"].config(
            command=lambda: self.show_frame(Frame1)
        )
        submenu1.options["Guassian NB Model"].config(
            command=lambda: self.show_frame(Frame2)
        )
        submenu1.options["Logistic Regression Model"].config(
            command=lambda: self.show_frame(Frame3)
        )

        submenu1.place(relx=0, rely=0.025, relwidth=1, relheight=0.3)

        submenu2 = SidebarSubMenu(self.submenu_frame,
                                  sub_menu_heading='Dangerous Driver Prediction',
                                  sub_menu_options=["MLP Classifier Model", "Gradient Boosting Model", "Random Forest Classifier"]
                                  )
        submenu2.options["MLP Classifier Model"].config(
            command=lambda: self.show_frame(AnotherFrame1)
        )
        submenu2.options["Gradient Boosting Model"].config(
            command=lambda: self.show_frame(AnotherFrame2)
        )
        submenu2.options["Random Forest Classifier"].config(
            command=lambda: self.show_frame(AnotherFrame3)
        )
        submenu2.place(relx=0, rely=0.35, relwidth=1, relheight=0.3)

        container = tk.Frame(self)
        container.config(highlightbackground="#808080", highlightthickness=0.5)
        container.place(relx=0.3, rely=0.1, relwidth=0.7, relheight=0.9)

        self.frames = {}

        for F in (Frame1,
                  Frame2,
                  Frame3,
                  AnotherFrame1,
                  AnotherFrame2,
                  AnotherFrame3,
                  ):

            frame = F(container, self)
            self.frames[F] = frame
            frame.place(relx=0, rely=0, relwidth=1, relheight=1)
        self.show_frame(Frame1)

    def show_frame(self, cont):
        current_frame = self.frames[cont]
        current_frame.reset()
        frame = self.frames[cont]
        frame.tkraise()


class Frame1(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)

        label = tk.Label(self, text='MLP Classifider', font=("Arial", 15))
        label.pack()

        dob_label = tk.Label(self, text='Date of Birth (Year):')
        dob_label.pack()
        self.dob_var = tk.StringVar()
        dob_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.dob_var)
        dob_entry.pack()

        exp_label = tk.Label(self, text='Years of Experience:')
        exp_label.pack()
        self.exp_var = tk.StringVar()
        exp_entry = tk.Spinbox(self, from_=0, to=100, textvariable=self.exp_var)
        exp_entry.pack()

        gender_label = tk.Label(self, text='Gender:')
        gender_label.pack()
        self.gender_var = tk.StringVar()
        gender_combobox = ttk.Combobox(self, textvariable=self.gender_var, values=['Male', 'Female'])
        gender_combobox.pack()

        car_brand_label = tk.Label(self, text='Car Brand:')
        car_brand_label.pack()
        self.car_brand_var = tk.StringVar()
        car_brand_combobox = ttk.Combobox(self, textvariable=self.car_brand_var,
                                         values=['Audi', 'BMW', 'Honda', 'Hyundai', 'Kia', 'Lexus', 'Mazda',
                                                 'Mercedes-Benz', 'Mitsubishi', 'Nissan', 'Toyota', 'Volkswagen'])
        car_brand_combobox.pack()

        car_model_year_label = tk.Label(self, text='Car Model Year:')
        car_model_year_label.pack()
        self.car_model_year_var = tk.StringVar()
        car_model_year_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.car_model_year_var)
        car_model_year_entry.pack()

        rating_label = tk.Label(self, text='Driver Rating:')
        rating_label.pack()
        self.rating_var = tk.StringVar()
        rating_entry = tk.Entry(self, textvariable=self.rating_var, validate="key",
                                validatecommand=(self.register(self.validate_rating), "%P"))
        rating_entry.pack()
        
        acceleration_label = tk.Label(self, text='Acceleration:')
        acceleration_label.pack()
        self.acceleration_var = tk.StringVar()
        acceleration_entry = tk.Entry(self, textvariable=self.acceleration_var, validate="key",
                                      validatecommand=(self.register(self.validate_acceleration), "%P"))
        acceleration_entry.pack()

        # New entry for seconds (limited to 500)
        seconds_label = tk.Label(self, text='Seconds:')
        seconds_label.pack()
        self.seconds_var = tk.StringVar()
        seconds_entry = tk.Entry(self, textvariable=self.seconds_var, validate="key",
                                 validatecommand=(self.register(self.validate_seconds), "%P"))
        seconds_entry.pack()
        
        speed_label = tk.Label(self, text='Speed:')
        speed_label.pack()
        self.speed_var = tk.StringVar()
        speed_entry = tk.Entry(self, textvariable=self.speed_var, validate="key",
                               validatecommand=(self.register(self.validate_speed), "%P"))
        speed_entry.pack()


        # Entry widget for file path
        file_entry_label = tk.Label(self, text='Excel File Path:')
        file_entry_label.pack()
        self.file_path_var = tk.StringVar()
        file_entry = tk.Entry(self, textvariable=self.file_path_var)
        file_entry.pack()

        # Button to trigger user input predictions
        predict_button = tk.Button(self, text='Predict', command=self.predict)
        predict_button.pack()

        # Button to trigger Excel predictions
        predict_excel_button = tk.Button(self, text='Predict from Excel', command=self.predict_from_excel)
        predict_excel_button.pack()

        self.validation_message = tk.Label(self, text='', fg='red')
        self.validation_message.pack()

        self.result_label = tk.Label(self, text='', fg='green')
        self.result_label.pack()

        self.mlp_model = joblib.load('models/mlpclassifier.pkl')

    def reset(self):
        self.dob_var.set('')
        self.exp_var.set('')
        self.gender_var.set('')
        self.car_brand_var.set('')
        self.car_model_year_var.set('')
        self.rating_var.set('')
        self.file_path_var.set('')
        self.validation_message.config(text='')
        self.result_label.config(text='')

    def predict(self):
        if self.validate_inputs():
            input_features = self.get_input_features()
            prediction_result = self.mlp_model.predict([input_features])
            dangerous_label = "Dangerous" if prediction_result[0] == 1 else "Not Dangerous"
            color = 'red' if prediction_result[0] == 1 else 'green'
            self.result_label.config(text=f"Prediction result: {dangerous_label}", fg=color)

    def get_input_features(self):
        gender_mapping = {'Female': 0, 'Male': 1}
        car_brand_mapping = {'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                             'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                             'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11}
        dob = int(self.dob_var.get())
        exp = int(self.exp_var.get())
        gender = gender_mapping.get(self.gender_var.get(), -1)
        car_brand = car_brand_mapping.get(self.car_brand_var.get(), -1)
        car_model_year = int(self.car_model_year_var.get())
        rating = float(self.rating_var.get())
        acceleration = float(self.acceleration_var.get()) if self.acceleration_var.get() else 0.0
        seconds = float(self.seconds_var.get()) if self.seconds_var.get() else 0.0
        speed = float(self.speed_var.get()) if self.speed_var.get() else 0.0
        
        
        input_features = np.array([dob, exp, gender, car_brand, car_model_year, rating, acceleration, seconds, speed])

        return input_features


    def validate_inputs(self):
        if not self.validate_rating(self.rating_var.get()):
            return False

        return True

    def validate_rating(self, new_value):
        if new_value == "":
            self.validation_message.config(text='')
            return True

        try:
            rating = float(new_value)
            if 0 <= rating <= 5:
                self.validation_message.config(text='')
                return True
            else:
                self.validation_message.config(text='Rating must be between 0 and 5')
                return False
        except ValueError:
            self.validation_message.config(text='Invalid input')
            return False
    def validate_acceleration(self, new_value):
        if new_value == "":
            return True

        try:
            acceleration = float(new_value)
            return True
        except ValueError:
            return False

    def validate_seconds(self, new_value):
        if new_value == "":
            return True

        try:
            seconds = float(new_value)
            return 0 <= seconds <= 500
        except ValueError:
            return False
        
    def validate_speed(self, new_value):
        if new_value == "":
            return True

        try:
            speed = float(new_value)
            return True
        except ValueError:
            return False


    def predict_from_excel(self):
        # Get the file path from the entry widget
        file_path = self.file_path_var.get()

        if file_path:
            try:
                # Read input data from Excel file
                input_data = pd.read_excel(file_path)

                # Validate and preprocess input data
                if self.validate_excel_data(input_data):
                    # Convert categorical columns to numeric using mappings
                    input_data['gender'] = input_data['gender'].map({'Female': 0, 'Male': 1})
                    input_data['car_brand'] = input_data['car_brand'].map({'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                                                                         'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                                                                         'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11})

                    # Perform predictions using the loaded model
                    predictions = self.mlp_model.predict(input_data)

                    # Display prediction results
                    self.display_excel_predictions(input_data, predictions)
            except Exception as e:
                print(f"Error reading Excel file: {str(e)}")

    def validate_excel_data(self, input_data):
        # Check if the input data has the expected format and structure
        if isinstance(input_data, pd.DataFrame):
            return True
        else:
            return False

    def display_excel_predictions(self, input_data, predictions):
        # Create a new Toplevel window to display the results
        result_window = tk.Toplevel(self)
        result_window.title("Excel Predictions")

        # Create a text widget to display predictions
        result_text = tk.Text(result_window, wrap="none", height=10, width=50)
        result_text.pack(padx=10, pady=10)

        # Insert headers
        result_text.insert("1.0", "ID\tPrediction\n")
        result_text.insert("2.0", "-" * 20 + "\n")

        # Insert predictions for each row
        for i, prediction in enumerate(predictions):
            # Use index as ID if no 'ID' column is present in the input_data
            id_value = input_data.iloc[i]['ID'] if 'ID' in input_data.columns else i + 1
            result_text.insert(tk.END, f"{id_value}\t{'Dangerous' if prediction == 1 else 'Not Dangerous'}\n")

        # Disable text editing
        result_text.config(state=tk.DISABLED)

class Frame2(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)

        label = tk.Label(self, text='Guassian NB Model', font=("Arial", 15))
        label.pack()

        dob_label = tk.Label(self, text='Date of Birth (Year):')
        dob_label.pack()
        self.dob_var = tk.StringVar()
        dob_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.dob_var)
        dob_entry.pack()

        exp_label = tk.Label(self, text='Years of Experience:')
        exp_label.pack()
        self.exp_var = tk.StringVar()
        exp_entry = tk.Spinbox(self, from_=0, to=100, textvariable=self.exp_var)
        exp_entry.pack()

        gender_label = tk.Label(self, text='Gender:')
        gender_label.pack()
        self.gender_var = tk.StringVar()
        gender_combobox = ttk.Combobox(self, textvariable=self.gender_var, values=['Male', 'Female'])
        gender_combobox.pack()

        car_brand_label = tk.Label(self, text='Car Brand:')
        car_brand_label.pack()
        self.car_brand_var = tk.StringVar()
        car_brand_combobox = ttk.Combobox(self, textvariable=self.car_brand_var,
                                         values=['Audi', 'BMW', 'Honda', 'Hyundai', 'Kia', 'Lexus', 'Mazda',
                                                 'Mercedes-Benz', 'Mitsubishi', 'Nissan', 'Toyota', 'Volkswagen'])
        car_brand_combobox.pack()

        car_model_year_label = tk.Label(self, text='Car Model Year:')
        car_model_year_label.pack()
        self.car_model_year_var = tk.StringVar()
        car_model_year_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.car_model_year_var)
        car_model_year_entry.pack()

        rating_label = tk.Label(self, text='Driver Rating:')
        rating_label.pack()
        self.rating_var = tk.StringVar()
        rating_entry = tk.Entry(self, textvariable=self.rating_var, validate="key",
                                validatecommand=(self.register(self.validate_rating), "%P"))
        rating_entry.pack()
        
        acceleration_label = tk.Label(self, text='Acceleration:')
        acceleration_label.pack()
        self.acceleration_var = tk.StringVar()
        acceleration_entry = tk.Entry(self, textvariable=self.acceleration_var, validate="key",
                                      validatecommand=(self.register(self.validate_acceleration), "%P"))
        acceleration_entry.pack()

        # New entry for seconds (limited to 500)
        seconds_label = tk.Label(self, text='Seconds:')
        seconds_label.pack()
        self.seconds_var = tk.StringVar()
        seconds_entry = tk.Entry(self, textvariable=self.seconds_var, validate="key",
                                 validatecommand=(self.register(self.validate_seconds), "%P"))
        seconds_entry.pack()
        
        speed_label = tk.Label(self, text='Speed:')
        speed_label.pack()
        self.speed_var = tk.StringVar()
        speed_entry = tk.Entry(self, textvariable=self.speed_var, validate="key",
                               validatecommand=(self.register(self.validate_speed), "%P"))
        speed_entry.pack()


        # Entry widget for file path
        file_entry_label = tk.Label(self, text='Excel File Path:')
        file_entry_label.pack()
        self.file_path_var = tk.StringVar()
        file_entry = tk.Entry(self, textvariable=self.file_path_var)
        file_entry.pack()

        # Button to trigger user input predictions
        predict_button = tk.Button(self, text='Predict', command=self.predict)
        predict_button.pack()

        # Button to trigger Excel predictions
        predict_excel_button = tk.Button(self, text='Predict from Excel', command=self.predict_from_excel)
        predict_excel_button.pack()

        self.validation_message = tk.Label(self, text='', fg='red')
        self.validation_message.pack()

        self.result_label = tk.Label(self, text='', fg='green')
        self.result_label.pack()

        self.mlp_model = joblib.load('models/gaussianNB.pkl')

    def reset(self):
        self.dob_var.set('')
        self.exp_var.set('')
        self.gender_var.set('')
        self.car_brand_var.set('')
        self.car_model_year_var.set('')
        self.rating_var.set('')
        self.file_path_var.set('')
        self.validation_message.config(text='')
        self.result_label.config(text='')

    def predict(self):
        if self.validate_inputs():
            input_features = self.get_input_features()
            prediction_result = self.mlp_model.predict([input_features])
            dangerous_label = "Dangerous" if prediction_result[0] == 1 else "Not Dangerous"
            color = 'red' if prediction_result[0] == 1 else 'green'
            self.result_label.config(text=f"Prediction result: {dangerous_label}", fg=color)

    def get_input_features(self):
        gender_mapping = {'Female': 0, 'Male': 1}
        car_brand_mapping = {'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                             'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                             'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11}
        dob = int(self.dob_var.get())
        exp = int(self.exp_var.get())
        gender = gender_mapping.get(self.gender_var.get(), -1)
        car_brand = car_brand_mapping.get(self.car_brand_var.get(), -1)
        car_model_year = int(self.car_model_year_var.get())
        rating = float(self.rating_var.get())
        acceleration = float(self.acceleration_var.get()) if self.acceleration_var.get() else 0.0
        seconds = float(self.seconds_var.get()) if self.seconds_var.get() else 0.0
        speed = float(self.speed_var.get()) if self.speed_var.get() else 0.0
        
        
        input_features = np.array([dob, exp, gender, car_brand, car_model_year, rating, acceleration, seconds, speed])

        return input_features


    def validate_inputs(self):
        if not self.validate_rating(self.rating_var.get()):
            return False

        return True

    def validate_rating(self, new_value):
        if new_value == "":
            self.validation_message.config(text='')
            return True

        try:
            rating = float(new_value)
            if 0 <= rating <= 5:
                self.validation_message.config(text='')
                return True
            else:
                self.validation_message.config(text='Rating must be between 0 and 5')
                return False
        except ValueError:
            self.validation_message.config(text='Invalid input')
            return False
    def validate_acceleration(self, new_value):
        if new_value == "":
            return True

        try:
            acceleration = float(new_value)
            return True
        except ValueError:
            return False

    def validate_seconds(self, new_value):
        if new_value == "":
            return True

        try:
            seconds = float(new_value)
            return 0 <= seconds <= 500
        except ValueError:
            return False
        
    def validate_speed(self, new_value):
        if new_value == "":
            return True

        try:
            speed = float(new_value)
            return True
        except ValueError:
            return False


    def predict_from_excel(self):
        # Get the file path from the entry widget
        file_path = self.file_path_var.get()

        if file_path:
            try:
                # Read input data from Excel file
                input_data = pd.read_excel(file_path)

                # Validate and preprocess input data
                if self.validate_excel_data(input_data):
                    # Convert categorical columns to numeric using mappings
                    input_data['gender'] = input_data['gender'].map({'Female': 0, 'Male': 1})
                    input_data['car_brand'] = input_data['car_brand'].map({'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                                                                         'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                                                                         'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11})

                    # Perform predictions using the loaded model
                    predictions = self.mlp_model.predict(input_data)

                    # Display prediction results
                    self.display_excel_predictions(input_data, predictions)
            except Exception as e:
                print(f"Error reading Excel file: {str(e)}")

    def validate_excel_data(self, input_data):
        # Check if the input data has the expected format and structure
        if isinstance(input_data, pd.DataFrame):
            return True
        else:
            return False

    def display_excel_predictions(self, input_data, predictions):
        # Create a new Toplevel window to display the results
        result_window = tk.Toplevel(self)
        result_window.title("Excel Predictions")

        # Create a text widget to display predictions
        result_text = tk.Text(result_window, wrap="none", height=10, width=50)
        result_text.pack(padx=10, pady=10)

        # Insert headers
        result_text.insert("1.0", "ID\tPrediction\n")
        result_text.insert("2.0", "-" * 20 + "\n")

        # Insert predictions for each row
        for i, prediction in enumerate(predictions):
            # Use index as ID if no 'ID' column is present in the input_data
            id_value = input_data.iloc[i]['ID'] if 'ID' in input_data.columns else i + 1
            result_text.insert(tk.END, f"{id_value}\t{'Dangerous' if prediction == 1 else 'Not Dangerous'}\n")

        # Disable text editing
        result_text.config(state=tk.DISABLED)
class Frame3(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)

        label = tk.Label(self, text='Logistic Regression Model', font=("Arial", 15))
        label.pack()

        dob_label = tk.Label(self, text='Date of Birth (Year):')
        dob_label.pack()
        self.dob_var = tk.StringVar()
        dob_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.dob_var)
        dob_entry.pack()

        exp_label = tk.Label(self, text='Years of Experience:')
        exp_label.pack()
        self.exp_var = tk.StringVar()
        exp_entry = tk.Spinbox(self, from_=0, to=100, textvariable=self.exp_var)
        exp_entry.pack()

        gender_label = tk.Label(self, text='Gender:')
        gender_label.pack()
        self.gender_var = tk.StringVar()
        gender_combobox = ttk.Combobox(self, textvariable=self.gender_var, values=['Male', 'Female'])
        gender_combobox.pack()

        car_brand_label = tk.Label(self, text='Car Brand:')
        car_brand_label.pack()
        self.car_brand_var = tk.StringVar()
        car_brand_combobox = ttk.Combobox(self, textvariable=self.car_brand_var,
                                         values=['Audi', 'BMW', 'Honda', 'Hyundai', 'Kia', 'Lexus', 'Mazda',
                                                 'Mercedes-Benz', 'Mitsubishi', 'Nissan', 'Toyota', 'Volkswagen'])
        car_brand_combobox.pack()

        car_model_year_label = tk.Label(self, text='Car Model Year:')
        car_model_year_label.pack()
        self.car_model_year_var = tk.StringVar()
        car_model_year_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.car_model_year_var)
        car_model_year_entry.pack()

        rating_label = tk.Label(self, text='Driver Rating:')
        rating_label.pack()
        self.rating_var = tk.StringVar()
        rating_entry = tk.Entry(self, textvariable=self.rating_var, validate="key",
                                validatecommand=(self.register(self.validate_rating), "%P"))
        rating_entry.pack()
        
        acceleration_label = tk.Label(self, text='Acceleration:')
        acceleration_label.pack()
        self.acceleration_var = tk.StringVar()
        acceleration_entry = tk.Entry(self, textvariable=self.acceleration_var, validate="key",
                                      validatecommand=(self.register(self.validate_acceleration), "%P"))
        acceleration_entry.pack()

        # New entry for seconds (limited to 500)
        seconds_label = tk.Label(self, text='Seconds:')
        seconds_label.pack()
        self.seconds_var = tk.StringVar()
        seconds_entry = tk.Entry(self, textvariable=self.seconds_var, validate="key",
                                 validatecommand=(self.register(self.validate_seconds), "%P"))
        seconds_entry.pack()
        
        speed_label = tk.Label(self, text='Speed:')
        speed_label.pack()
        self.speed_var = tk.StringVar()
        speed_entry = tk.Entry(self, textvariable=self.speed_var, validate="key",
                               validatecommand=(self.register(self.validate_speed), "%P"))
        speed_entry.pack()


        # Entry widget for file path
        file_entry_label = tk.Label(self, text='Excel File Path:')
        file_entry_label.pack()
        self.file_path_var = tk.StringVar()
        file_entry = tk.Entry(self, textvariable=self.file_path_var)
        file_entry.pack()

        # Button to trigger user input predictions
        predict_button = tk.Button(self, text='Predict', command=self.predict)
        predict_button.pack()

        # Button to trigger Excel predictions
        predict_excel_button = tk.Button(self, text='Predict from Excel', command=self.predict_from_excel)
        predict_excel_button.pack()

        self.validation_message = tk.Label(self, text='', fg='red')
        self.validation_message.pack()

        self.result_label = tk.Label(self, text='', fg='green')
        self.result_label.pack()

        self.mlp_model = joblib.load('models/logreg.pkl')

    def reset(self):
        self.dob_var.set('')
        self.exp_var.set('')
        self.gender_var.set('')
        self.car_brand_var.set('')
        self.car_model_year_var.set('')
        self.rating_var.set('')
        self.file_path_var.set('')
        self.validation_message.config(text='')
        self.result_label.config(text='')

    def predict(self):
        if self.validate_inputs():
            input_features = self.get_input_features()
            prediction_result = self.mlp_model.predict([input_features])
            dangerous_label = "Dangerous" if prediction_result[0] == 1 else "Not Dangerous"
            color = 'red' if prediction_result[0] == 1 else 'green'
            self.result_label.config(text=f"Prediction result: {dangerous_label}", fg=color)

    def get_input_features(self):
        gender_mapping = {'Female': 0, 'Male': 1}
        car_brand_mapping = {'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                             'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                             'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11}
        dob = int(self.dob_var.get())
        exp = int(self.exp_var.get())
        gender = gender_mapping.get(self.gender_var.get(), -1)
        car_brand = car_brand_mapping.get(self.car_brand_var.get(), -1)
        car_model_year = int(self.car_model_year_var.get())
        rating = float(self.rating_var.get())
        acceleration = float(self.acceleration_var.get()) if self.acceleration_var.get() else 0.0
        seconds = float(self.seconds_var.get()) if self.seconds_var.get() else 0.0
        speed = float(self.speed_var.get()) if self.speed_var.get() else 0.0
        
        
        input_features = np.array([dob, exp, gender, car_brand, car_model_year, rating, acceleration, seconds, speed])

        return input_features


    def validate_inputs(self):
        if not self.validate_rating(self.rating_var.get()):
            return False

        return True

    def validate_rating(self, new_value):
        if new_value == "":
            self.validation_message.config(text='')
            return True

        try:
            rating = float(new_value)
            if 0 <= rating <= 5:
                self.validation_message.config(text='')
                return True
            else:
                self.validation_message.config(text='Rating must be between 0 and 5')
                return False
        except ValueError:
            self.validation_message.config(text='Invalid input')
            return False
    def validate_acceleration(self, new_value):
        if new_value == "":
            return True

        try:
            acceleration = float(new_value)
            return True
        except ValueError:
            return False

    def validate_seconds(self, new_value):
        if new_value == "":
            return True

        try:
            seconds = float(new_value)
            return 0 <= seconds <= 500
        except ValueError:
            return False
        
    def validate_speed(self, new_value):
        if new_value == "":
            return True

        try:
            speed = float(new_value)
            return True
        except ValueError:
            return False


    def predict_from_excel(self):
        # Get the file path from the entry widget
        file_path = self.file_path_var.get()

        if file_path:
            try:
                # Read input data from Excel file
                input_data = pd.read_excel(file_path)

                # Validate and preprocess input data
                if self.validate_excel_data(input_data):
                    # Convert categorical columns to numeric using mappings
                    input_data['gender'] = input_data['gender'].map({'Female': 0, 'Male': 1})
                    input_data['car_brand'] = input_data['car_brand'].map({'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                                                                         'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                                                                         'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11})

                    # Perform predictions using the loaded model
                    predictions = self.mlp_model.predict(input_data)

                    # Display prediction results
                    self.display_excel_predictions(input_data, predictions)
            except Exception as e:
                print(f"Error reading Excel file: {str(e)}")

    def validate_excel_data(self, input_data):
        # Check if the input data has the expected format and structure
        if isinstance(input_data, pd.DataFrame):
            return True
        else:
            return False

    def display_excel_predictions(self, input_data, predictions):
        # Create a new Toplevel window to display the results
        result_window = tk.Toplevel(self)
        result_window.title("Excel Predictions")

        # Create a text widget to display predictions
        result_text = tk.Text(result_window, wrap="none", height=10, width=50)
        result_text.pack(padx=10, pady=10)

        # Insert headers
        result_text.insert("1.0", "ID\tPrediction\n")
        result_text.insert("2.0", "-" * 20 + "\n")

        # Insert predictions for each row
        for i, prediction in enumerate(predictions):
            # Use index as ID if no 'ID' column is present in the input_data
            id_value = input_data.iloc[i]['ID'] if 'ID' in input_data.columns else i + 1
            result_text.insert(tk.END, f"{id_value}\t{'Dangerous' if prediction == 1 else 'Not Dangerous'}\n")

        # Disable text editing
        result_text.config(state=tk.DISABLED)
class AnotherFrame1(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)

        label = tk.Label(self, text='MLP Classifider', font=("Arial", 15))
        label.pack()

        dob_label = tk.Label(self, text='Date of Birth (Year):')
        dob_label.pack()
        self.dob_var = tk.StringVar()
        dob_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.dob_var)
        dob_entry.pack()

        exp_label = tk.Label(self, text='Years of Experience:')
        exp_label.pack()
        self.exp_var = tk.StringVar()
        exp_entry = tk.Spinbox(self, from_=0, to=100, textvariable=self.exp_var)
        exp_entry.pack()

        gender_label = tk.Label(self, text='Gender:')
        gender_label.pack()
        self.gender_var = tk.StringVar()
        gender_combobox = ttk.Combobox(self, textvariable=self.gender_var, values=['Male', 'Female'])
        gender_combobox.pack()

        car_brand_label = tk.Label(self, text='Car Brand:')
        car_brand_label.pack()
        self.car_brand_var = tk.StringVar()
        car_brand_combobox = ttk.Combobox(self, textvariable=self.car_brand_var,
                                         values=['Audi', 'BMW', 'Honda', 'Hyundai', 'Kia', 'Lexus', 'Mazda',
                                                 'Mercedes-Benz', 'Mitsubishi', 'Nissan', 'Toyota', 'Volkswagen'])
        car_brand_combobox.pack()

        car_model_year_label = tk.Label(self, text='Car Model Year:')
        car_model_year_label.pack()
        self.car_model_year_var = tk.StringVar()
        car_model_year_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.car_model_year_var)
        car_model_year_entry.pack()

        rating_label = tk.Label(self, text='Driver Rating:')
        rating_label.pack()
        self.rating_var = tk.StringVar()
        rating_entry = tk.Entry(self, textvariable=self.rating_var, validate="key",
                                validatecommand=(self.register(self.validate_rating), "%P"))
        rating_entry.pack()

        # Entry widget for file path
        file_entry_label = tk.Label(self, text='Excel File Path:')
        file_entry_label.pack()
        self.file_path_var = tk.StringVar()
        file_entry = tk.Entry(self, textvariable=self.file_path_var)
        file_entry.pack()

        # Button to trigger user input predictions
        predict_button = tk.Button(self, text='Predict', command=self.predict)
        predict_button.pack()

        # Button to trigger Excel predictions
        predict_excel_button = tk.Button(self, text='Predict from Excel', command=self.predict_from_excel)
        predict_excel_button.pack()

        self.validation_message = tk.Label(self, text='', fg='red')
        self.validation_message.pack()

        self.result_label = tk.Label(self, text='', fg='green')
        self.result_label.pack()

        self.mlp_model = joblib.load('models/mlp.pkl')

    def reset(self):
        self.dob_var.set('')
        self.exp_var.set('')
        self.gender_var.set('')
        self.car_brand_var.set('')
        self.car_model_year_var.set('')
        self.rating_var.set('')
        self.file_path_var.set('')
        self.validation_message.config(text='')
        self.result_label.config(text='')

    def predict(self):
        if self.validate_inputs():
            input_features = self.get_input_features()
            prediction_result = self.mlp_model.predict([input_features])
            dangerous_label = "Dangerous" if prediction_result[0] == 1 else "Not Dangerous"
            color = 'red' if prediction_result[0] == 1 else 'green'
            self.result_label.config(text=f"Prediction result: {dangerous_label}", fg=color)

    def get_input_features(self):
        gender_mapping = {'Female': 0, 'Male': 1}
        car_brand_mapping = {'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                             'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                             'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11}
        dob = int(self.dob_var.get())
        exp = int(self.exp_var.get())
        gender = gender_mapping.get(self.gender_var.get(), -1)
        car_brand = car_brand_mapping.get(self.car_brand_var.get(), -1)
        car_model_year = int(self.car_model_year_var.get())
        rating = float(self.rating_var.get())

        input_features = np.array([dob, exp, gender, car_brand, car_model_year, rating])

        return input_features

    def validate_inputs(self):
        if not self.validate_rating(self.rating_var.get()):
            return False

        return True

    def validate_rating(self, new_value):
        if new_value == "":
            self.validation_message.config(text='')
            return True

        try:
            rating = float(new_value)
            if 0 <= rating <= 5:
                self.validation_message.config(text='')
                return True
            else:
                self.validation_message.config(text='Rating must be between 0 and 5')
                return False
        except ValueError:
            self.validation_message.config(text='Invalid input')
            return False

    def predict_from_excel(self):
        # Get the file path from the entry widget
        file_path = self.file_path_var.get()

        if file_path:
            try:
                # Read input data from Excel file
                input_data = pd.read_excel(file_path)

                # Validate and preprocess input data
                if self.validate_excel_data(input_data):
                    # Convert categorical columns to numeric using mappings
                    input_data['gender'] = input_data['gender'].map({'Female': 0, 'Male': 1})
                    input_data['car_brand'] = input_data['car_brand'].map({'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                                                                         'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                                                                         'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11})

                    # Perform predictions using the loaded model
                    predictions = self.mlp_model.predict(input_data)

                    # Display prediction results
                    self.display_excel_predictions(input_data, predictions)
            except Exception as e:
                print(f"Error reading Excel file: {str(e)}")

    def validate_excel_data(self, input_data):
        # Check if the input data has the expected format and structure
        if isinstance(input_data, pd.DataFrame):
            return True
        else:
            return False

    def display_excel_predictions(self, input_data, predictions):
        # Create a new Toplevel window to display the results
        result_window = tk.Toplevel(self)
        result_window.title("Excel Predictions")

        # Create a text widget to display predictions
        result_text = tk.Text(result_window, wrap="none", height=10, width=50)
        result_text.pack(padx=10, pady=10)

        # Insert headers
        result_text.insert("1.0", "ID\tPrediction\n")
        result_text.insert("2.0", "-" * 20 + "\n")

        # Insert predictions for each row
        for i, prediction in enumerate(predictions):
            # Use index as ID if no 'ID' column is present in the input_data
            id_value = input_data.iloc[i]['ID'] if 'ID' in input_data.columns else i + 1
            result_text.insert(tk.END, f"{id_value}\t{'Dangerous' if prediction == 1 else 'Not Dangerous'}\n")

        # Disable text editing
        result_text.config(state=tk.DISABLED)
        
class AnotherFrame2(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)

        label = tk.Label(self, text='Gradient Boosting', font=("Arial", 15))
        label.pack()

        dob_label = tk.Label(self, text='Date of Birth (Year):')
        dob_label.pack()
        self.dob_var = tk.StringVar()
        dob_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.dob_var)
        dob_entry.pack()

        exp_label = tk.Label(self, text='Years of Experience:')
        exp_label.pack()
        self.exp_var = tk.StringVar()
        exp_entry = tk.Spinbox(self, from_=0, to=100, textvariable=self.exp_var)
        exp_entry.pack()

        gender_label = tk.Label(self, text='Gender:')
        gender_label.pack()
        self.gender_var = tk.StringVar()
        gender_combobox = ttk.Combobox(self, textvariable=self.gender_var, values=['Male', 'Female'])
        gender_combobox.pack()

        car_brand_label = tk.Label(self, text='Car Brand:')
        car_brand_label.pack()
        self.car_brand_var = tk.StringVar()
        car_brand_combobox = ttk.Combobox(self, textvariable=self.car_brand_var,
                                         values=['Audi', 'BMW', 'Honda', 'Hyundai', 'Kia', 'Lexus', 'Mazda',
                                                 'Mercedes-Benz', 'Mitsubishi', 'Nissan', 'Toyota', 'Volkswagen'])
        car_brand_combobox.pack()

        car_model_year_label = tk.Label(self, text='Car Model Year:')
        car_model_year_label.pack()
        self.car_model_year_var = tk.StringVar()
        car_model_year_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.car_model_year_var)
        car_model_year_entry.pack()

        rating_label = tk.Label(self, text='Driver Rating:')
        rating_label.pack()
        self.rating_var = tk.StringVar()
        rating_entry = tk.Entry(self, textvariable=self.rating_var, validate="key",
                                validatecommand=(self.register(self.validate_rating), "%P"))
        rating_entry.pack()

        # Entry widget for file path
        file_entry_label = tk.Label(self, text='Excel File Path:')
        file_entry_label.pack()
        self.file_path_var = tk.StringVar()
        file_entry = tk.Entry(self, textvariable=self.file_path_var)
        file_entry.pack()

        # Button to trigger user input predictions
        predict_button = tk.Button(self, text='Predict', command=self.predict)
        predict_button.pack()

        # Button to trigger Excel predictions
        predict_excel_button = tk.Button(self, text='Predict from Excel', command=self.predict_from_excel)
        predict_excel_button.pack()

        self.validation_message = tk.Label(self, text='', fg='red')
        self.validation_message.pack()

        self.result_label = tk.Label(self, text='', fg='green')
        self.result_label.pack()

        self.mlp_model = joblib.load('models/gb.pkl')

    def reset(self):
        self.dob_var.set('')
        self.exp_var.set('')
        self.gender_var.set('')
        self.car_brand_var.set('')
        self.car_model_year_var.set('')
        self.rating_var.set('')
        self.file_path_var.set('')
        self.validation_message.config(text='')
        self.result_label.config(text='')

    def predict(self):
        if self.validate_inputs():
            input_features = self.get_input_features()
            prediction_result = self.mlp_model.predict([input_features])
            dangerous_label = "Dangerous" if prediction_result[0] == 1 else "Not Dangerous"
            color = 'red' if prediction_result[0] == 1 else 'green'
            self.result_label.config(text=f"Prediction result: {dangerous_label}", fg=color)

    def get_input_features(self):
        gender_mapping = {'Female': 0, 'Male': 1}
        car_brand_mapping = {'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                             'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                             'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11}
        dob = int(self.dob_var.get())
        exp = int(self.exp_var.get())
        gender = gender_mapping.get(self.gender_var.get(), -1)
        car_brand = car_brand_mapping.get(self.car_brand_var.get(), -1)
        car_model_year = int(self.car_model_year_var.get())
        rating = float(self.rating_var.get())

        input_features = np.array([dob, exp, gender, car_brand, car_model_year, rating])

        return input_features

    def validate_inputs(self):
        if not self.validate_rating(self.rating_var.get()):
            return False

        return True

    def validate_rating(self, new_value):
        if new_value == "":
            self.validation_message.config(text='')
            return True

        try:
            rating = float(new_value)
            if 0 <= rating <= 5:
                self.validation_message.config(text='')
                return True
            else:
                self.validation_message.config(text='Rating must be between 0 and 5')
                return False
        except ValueError:
            self.validation_message.config(text='Invalid input')
            return False

    def predict_from_excel(self):
        # Get the file path from the entry widget
        file_path = self.file_path_var.get()

        if file_path:
            try:
                # Read input data from Excel file
                input_data = pd.read_excel(file_path)

                # Validate and preprocess input data
                if self.validate_excel_data(input_data):
                    # Convert categorical columns to numeric using mappings
                    input_data['gender'] = input_data['gender'].map({'Female': 0, 'Male': 1})
                    input_data['car_brand'] = input_data['car_brand'].map({'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                                                                         'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                                                                         'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11})

                    # Perform predictions using the loaded model
                    predictions = self.mlp_model.predict(input_data)

                    # Display prediction results
                    self.display_excel_predictions(input_data, predictions)
            except Exception as e:
                print(f"Error reading Excel file: {str(e)}")

    def validate_excel_data(self, input_data):
        # Check if the input data has the expected format and structure
        if isinstance(input_data, pd.DataFrame):
            return True
        else:
            return False

    def display_excel_predictions(self, input_data, predictions):
        # Create a new Toplevel window to display the results
        result_window = tk.Toplevel(self)
        result_window.title("Excel Predictions")

        # Create a text widget to display predictions
        result_text = tk.Text(result_window, wrap="none", height=10, width=50)
        result_text.pack(padx=10, pady=10)

        # Insert headers
        result_text.insert("1.0", "ID\tPrediction\n")
        result_text.insert("2.0", "-" * 20 + "\n")

        # Insert predictions for each row
        for i, prediction in enumerate(predictions):
            # Use index as ID if no 'ID' column is present in the input_data
            id_value = input_data.iloc[i]['ID'] if 'ID' in input_data.columns else i + 1
            result_text.insert(tk.END, f"{id_value}\t{'Dangerous' if prediction == 1 else 'Not Dangerous'}\n")

        # Disable text editing
        result_text.config(state=tk.DISABLED)

class AnotherFrame3(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)

        label = tk.Label(self, text='Random Forest Classifier', font=("Arial", 15))
        label.pack()

        dob_label = tk.Label(self, text='Date of Birth (Year):')
        dob_label.pack()
        self.dob_var = tk.StringVar()
        dob_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.dob_var)
        dob_entry.pack()

        exp_label = tk.Label(self, text='Years of Experience:')
        exp_label.pack()
        self.exp_var = tk.StringVar()
        exp_entry = tk.Spinbox(self, from_=0, to=100, textvariable=self.exp_var)
        exp_entry.pack()

        gender_label = tk.Label(self, text='Gender:')
        gender_label.pack()
        self.gender_var = tk.StringVar()
        gender_combobox = ttk.Combobox(self, textvariable=self.gender_var, values=['Male', 'Female'])
        gender_combobox.pack()

        car_brand_label = tk.Label(self, text='Car Brand:')
        car_brand_label.pack()
        self.car_brand_var = tk.StringVar()
        car_brand_combobox = ttk.Combobox(self, textvariable=self.car_brand_var,
                                         values=['Audi', 'BMW', 'Honda', 'Hyundai', 'Kia', 'Lexus', 'Mazda',
                                                 'Mercedes-Benz', 'Mitsubishi', 'Nissan', 'Toyota', 'Volkswagen'])
        car_brand_combobox.pack()

        car_model_year_label = tk.Label(self, text='Car Model Year:')
        car_model_year_label.pack()
        self.car_model_year_var = tk.StringVar()
        car_model_year_entry = tk.Spinbox(self, from_=1950, to=2024, textvariable=self.car_model_year_var)
        car_model_year_entry.pack()

        rating_label = tk.Label(self, text='Driver Rating:')
        rating_label.pack()
        self.rating_var = tk.StringVar()
        rating_entry = tk.Entry(self, textvariable=self.rating_var, validate="key",
                                validatecommand=(self.register(self.validate_rating), "%P"))
        rating_entry.pack()

        # Entry widget for file path
        file_entry_label = tk.Label(self, text='Excel File Path:')
        file_entry_label.pack()
        self.file_path_var = tk.StringVar()
        file_entry = tk.Entry(self, textvariable=self.file_path_var)
        file_entry.pack()

        # Button to trigger user input predictions
        predict_button = tk.Button(self, text='Predict', command=self.predict)
        predict_button.pack()

        # Button to trigger Excel predictions
        predict_excel_button = tk.Button(self, text='Predict from Excel', command=self.predict_from_excel)
        predict_excel_button.pack()

        self.validation_message = tk.Label(self, text='', fg='red')
        self.validation_message.pack()

        self.result_label = tk.Label(self, text='', fg='green')
        self.result_label.pack()

        self.mlp_model = joblib.load('models/rf.pkl')

    def reset(self):
        self.dob_var.set('')
        self.exp_var.set('')
        self.gender_var.set('')
        self.car_brand_var.set('')
        self.car_model_year_var.set('')
        self.rating_var.set('')
        self.file_path_var.set('')
        self.validation_message.config(text='')
        self.result_label.config(text='')

    def predict(self):
        if self.validate_inputs():
            input_features = self.get_input_features()
            prediction_result = self.mlp_model.predict([input_features])
            dangerous_label = "Dangerous" if prediction_result[0] == 1 else "Not Dangerous"
            color = 'red' if prediction_result[0] == 1 else 'green'
            self.result_label.config(text=f"Prediction result: {dangerous_label}", fg=color)

    def get_input_features(self):
        gender_mapping = {'Female': 0, 'Male': 1}
        car_brand_mapping = {'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                             'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                             'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11}
        dob = int(self.dob_var.get())
        exp = int(self.exp_var.get())
        gender = gender_mapping.get(self.gender_var.get(), -1)
        car_brand = car_brand_mapping.get(self.car_brand_var.get(), -1)
        car_model_year = int(self.car_model_year_var.get())
        rating = float(self.rating_var.get())

        input_features = np.array([dob, exp, gender, car_brand, car_model_year, rating])

        return input_features

    def validate_inputs(self):
        if not self.validate_rating(self.rating_var.get()):
            return False

        return True

    def validate_rating(self, new_value):
        if new_value == "":
            self.validation_message.config(text='')
            return True

        try:
            rating = float(new_value)
            if 0 <= rating <= 5:
                self.validation_message.config(text='')
                return True
            else:
                self.validation_message.config(text='Rating must be between 0 and 5')
                return False
        except ValueError:
            self.validation_message.config(text='Invalid input')
            return False

    def predict_from_excel(self):
        # Get the file path from the entry widget
        file_path = self.file_path_var.get()

        if file_path:
            try:
                # Read input data from Excel file
                input_data = pd.read_excel(file_path)

                # Validate and preprocess input data
                if self.validate_excel_data(input_data):
                    # Convert categorical columns to numeric using mappings
                    input_data['gender'] = input_data['gender'].map({'Female': 0, 'Male': 1})
                    input_data['car_brand'] = input_data['car_brand'].map({'Audi': 0, 'BMW': 1, 'Honda': 2, 'Hyundai': 3,
                                                                         'Kia': 4, 'Lexus': 5, 'Mazda': 6, 'Mercedes-Benz': 7,
                                                                         'Mitsubishi': 8, 'Nissan': 9, 'Toyota': 10, 'Volkswagen': 11})

                    # Perform predictions using the loaded model
                    predictions = self.mlp_model.predict(input_data)

                    # Display prediction results
                    self.display_excel_predictions(input_data, predictions)
            except Exception as e:
                print(f"Error reading Excel file: {str(e)}")

    def validate_excel_data(self, input_data):
        # Check if the input data has the expected format and structure
        if isinstance(input_data, pd.DataFrame):
            # You can add additional validation logic here if needed
            return True
        else:
            return False

    def display_excel_predictions(self, input_data, predictions):
        # Create a new Toplevel window to display the results
        result_window = tk.Toplevel(self)
        result_window.title("Excel Predictions")

        # Create a text widget to display predictions
        result_text = tk.Text(result_window, wrap="none", height=10, width=50)
        result_text.pack(padx=10, pady=10)

        # Insert headers
        result_text.insert("1.0", "ID\tPrediction\n")
        result_text.insert("2.0", "-" * 20 + "\n")

        # Insert predictions for each row
        for i, prediction in enumerate(predictions):
            # Use index as ID if no 'ID' column is present in the input_data
            id_value = input_data.iloc[i]['ID'] if 'ID' in input_data.columns else i + 1
            result_text.insert(tk.END, f"{id_value}\t{'Dangerous' if prediction == 1 else 'Not Dangerous'}\n")

        # Disable text editing
        result_text.config(state=tk.DISABLED)
class SidebarSubMenu(tk.Frame):

    def __init__(self, parent, sub_menu_heading, sub_menu_options):
        tk.Frame.__init__(self, parent)
        self.config(bg=sidebar_color)
        self.sub_menu_heading_label = tk.Label(self,
                                               text=sub_menu_heading,
                                               bg=sidebar_color,
                                               fg="#333333",
                                               font=("Arial", 10)
                                               )
        self.sub_menu_heading_label.place(x=30, y=10, anchor="w")

        sub_menu_sep = ttk.Separator(self, orient='horizontal')
        sub_menu_sep.place(x=30, y=30, relwidth=0.8, anchor="w")

        self.options = {}
        for n, x in enumerate(sub_menu_options):
            self.options[x] = tk.Button(self,
                                        text=x,
                                        bg=sidebar_color,
                                        font=("Arial", 9, "bold"),
                                        bd=0,
                                        cursor='hand2',
                                        activebackground='#ffffff',
                                        )
            self.options[x].place(x=30, y=45 * (n + 1), anchor="w")


app = TkinterApp()
app.mainloop()
#Predict_Dangerous_Driver.xlsx

C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
